In [91]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as sps
import timeit

In [29]:
df = pd.read_csv('with_variables.csv', index_col = 0)

In [30]:
df.shape

(96397, 429)

## I. Train, test, oot

In [21]:
time = pd.read_excel('card transactions.xlsx')

In [22]:
time.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

In [23]:
time1 = time[(time.Date >= pd.to_datetime('2010-01-01') + dt.timedelta(14)) & (time.Date < pd.to_datetime('2010-11-01'))]

In [41]:
time1[(time1.Transtype == 'P') & (time1.Amount < 3102045)]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
3344,3345,5142289869,2010-01-15,602608969534,FISHER SCI ATL,GA,30091.0,P,333.47,0
3345,3346,5142148452,2010-01-15,5509006296254,FEDEX SHP 01/10/10 AB#,TN,38118.0,P,4.37,0
3346,3347,5142184598,2010-01-15,680655463,MC MASTER CARR SUPP,IL,60126.0,P,271.93,0
3347,3348,5142148452,2010-01-15,5509006296254,FEDEX SHP 01/10/10 AB#,TN,38118.0,P,3.62,0
3348,3349,5142195887,2010-01-15,4503082616100,OFFICE DEPOT #122,NC,27707.0,P,16.28,0
...,...,...,...,...,...,...,...,...,...,...
84294,84295,5142130739,2010-10-31,9108347680000,STAPLES NATIONAL #471,NJ,7606.0,P,299.77,0
84295,84296,5142219772,2010-10-31,6855293370648,PRESTIGE OFFICE PRODUCTS,NC,27705.0,P,609.34,0
84296,84297,5142257707,2010-10-31,300025852,AMERICAN SOCIETY OF AG,WI,53711.0,P,235.00,0
84297,84298,5142168022,2010-10-31,607900047334,MYSTIC LAKE CASINO,MN,55372.0,P,600.00,0


In [53]:
## shouldn't call it trn_tst, because I remove the first two weeks as well, just for feature selection.

trn_tst = df.loc[3345: 84299, :].copy()
trn_tst

,Fraud,Dow_Risk,Cardnum_day_since,Cardnum_count_0,Cardnum_avg_0,Cardnum_max_0,Cardnum_med_0,Cardnum_total_0,Cardnum_actual/avg_0,Cardnum_actual/max_0,...,merch_zip_count_0_by_30,merch_zip_count_1_by_7,merch_zip_count_1_by_14,merch_zip_count_1_by_30,merch_state_count_0_by_7,merch_state_count_0_by_14,merch_state_count_0_by_30,merch_state_count_1_by_7,merch_state_count_1_by_14,merch_state_count_1_by_30
Recnum,,,,,,,,,,,,,,,,,,,,,
3345,0,0.022419,1.0,1,333.470,333.47,333.470,333.47,1.000000,1.000000,...,1.000000,0.466667,0.466667,1.000000,0.466667,0.466667,1.000000,0.466667,0.466667,1.000000
3346,0,0.022419,1.0,1,4.370,4.37,4.370,4.37,1.000000,1.000000,...,0.098684,0.036649,0.046053,0.098684,0.036649,0.046053,0.098684,0.036649,0.046053,0.098684
3347,0,0.022419,1.0,1,271.930,271.93,271.930,271.93,1.000000,1.000000,...,2.307692,7.000000,3.230769,6.923077,2.333333,1.076923,2.307692,7.000000,3.230769,6.923077
3348,0,0.022419,0.0,2,3.995,4.37,3.995,7.99,0.906133,0.828375,...,0.196721,0.072917,0.091803,0.196721,0.072917,0.091803,0.196721,0.072917,0.091803,0.196721
3349,0,0.022419,14.0,1,16.280,16.28,16.280,16.28,1.000000,1.000000,...,6.000000,2.333333,2.800000,6.000000,2.333333,2.800000,6.000000,2.333333,2.800000,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84295,0,0.010444,23.0,1,299.770,299.77,299.770,299.77,1.000000,1.000000,...,2.608696,2.000000,1.866667,2.608696,2.000000,1.866667,2.608696,2.000000,1.866667,2.608696
84296,0,0.010444,0.0,2,394.660,609.34,394.660,789.32,1.543962,1.000000,...,6.000000,4.666667,8.000000,6.000000,4.666667,8.000000,6.000000,4.666667,8.000000,6.000000
84297,0,0.010444,3.0,1,235.000,235.00,235.000,235.00,1.000000,1.000000,...,30.000000,7.000000,14.000000,30.000000,7.000000,14.000000,30.000000,7.000000,14.000000,30.000000


In [54]:
trn_tst['Random'] = np.random.uniform(size = len(trn_tst))

In [55]:
trn_tst

,Fraud,Dow_Risk,Cardnum_day_since,Cardnum_count_0,Cardnum_avg_0,Cardnum_max_0,Cardnum_med_0,Cardnum_total_0,Cardnum_actual/avg_0,Cardnum_actual/max_0,...,merch_zip_count_1_by_7,merch_zip_count_1_by_14,merch_zip_count_1_by_30,merch_state_count_0_by_7,merch_state_count_0_by_14,merch_state_count_0_by_30,merch_state_count_1_by_7,merch_state_count_1_by_14,merch_state_count_1_by_30,Random
Recnum,,,,,,,,,,,,,,,,,,,,,
3345,0,0.022419,1.0,1,333.470,333.47,333.470,333.47,1.000000,1.000000,...,0.466667,0.466667,1.000000,0.466667,0.466667,1.000000,0.466667,0.466667,1.000000,0.221182
3346,0,0.022419,1.0,1,4.370,4.37,4.370,4.37,1.000000,1.000000,...,0.036649,0.046053,0.098684,0.036649,0.046053,0.098684,0.036649,0.046053,0.098684,0.940060
3347,0,0.022419,1.0,1,271.930,271.93,271.930,271.93,1.000000,1.000000,...,7.000000,3.230769,6.923077,2.333333,1.076923,2.307692,7.000000,3.230769,6.923077,0.775003
3348,0,0.022419,0.0,2,3.995,4.37,3.995,7.99,0.906133,0.828375,...,0.072917,0.091803,0.196721,0.072917,0.091803,0.196721,0.072917,0.091803,0.196721,0.399544
3349,0,0.022419,14.0,1,16.280,16.28,16.280,16.28,1.000000,1.000000,...,2.333333,2.800000,6.000000,2.333333,2.800000,6.000000,2.333333,2.800000,6.000000,0.059025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84295,0,0.010444,23.0,1,299.770,299.77,299.770,299.77,1.000000,1.000000,...,2.000000,1.866667,2.608696,2.000000,1.866667,2.608696,2.000000,1.866667,2.608696,0.654534
84296,0,0.010444,0.0,2,394.660,609.34,394.660,789.32,1.543962,1.000000,...,4.666667,8.000000,6.000000,4.666667,8.000000,6.000000,4.666667,8.000000,6.000000,0.821439
84297,0,0.010444,3.0,1,235.000,235.00,235.000,235.00,1.000000,1.000000,...,7.000000,14.000000,30.000000,7.000000,14.000000,30.000000,7.000000,14.000000,30.000000,0.368086


In [56]:
goods = trn_tst[trn_tst.Fraud == 0]
bads = trn_tst[trn_tst.Fraud == 1]

In [57]:
goods.shape, bads.shape

((79764, 430), (868, 430))

In [58]:
len(bads)/len(trn_tst)

0.010764956840956444

## II. Filter

### 2.1 KS score

In [59]:
ks_score = []
for column in trn_tst:
    ks_score.append(sps.ks_2samp(goods[column], bads[column])[0])

### 2.2 FDR

In [60]:
numbads = len(bads)
topRows = int(round(len(trn_tst)*0.03))

In [61]:
trn_tst['check_label'] = trn_tst.Fraud.values
fdr = []

for column in trn_tst.columns[:-1]:
    temp1 = trn_tst[[column, 'check_label']].sort_values(column, ascending = False).head(topRows)
    temp2 = trn_tst[[column, 'check_label']].sort_values(column, ascending = True).head(topRows)
    FDR1 = temp1.check_label.sum()/numbads
    FDR2 = temp2.check_label.sum()/numbads
    fdr.append(max(FDR1, FDR2))

In [62]:
trn_tst.drop('check_label', axis = 1, inplace = True)

### 2.3 Rankings

In [63]:
rank = pd.DataFrame({
    'Field': trn_tst.columns,
    'KS': ks_score,
    'FDR': fdr
})

In [64]:
rank['KS rank'] = rank.KS.rank()
rank['FDR rank'] = rank.FDR.rank()
rank['Average rank'] = (rank['KS rank'] + rank['FDR rank'])/2

In [65]:
rank.head()

,Field,KS,FDR,KS rank,FDR rank,Average rank
0,Fraud,1.000000,1.000000,430.0,430.0,430.00
1,Dow_Risk,0.134586,0.063364,29.0,143.5,86.25
2,Cardnum_day_since,0.291223,0.087558,200.0,202.5,201.25
3,Cardnum_count_0,0.339447,0.171659,238.0,250.0,244.00
4,Cardnum_avg_0,0.570021,0.328341,341.0,346.5,343.75


In [66]:
rank1 = rank.sort_values('KS rank', ascending = False)
rank1

,Field,KS,FDR,KS rank,FDR rank,Average rank
0,Fraud,1.000000,1.000000,430.0,430.0,430.00
199,card_zip_total_7,0.686347,0.639401,429.0,428.0,428.50
144,card_merch_total_7,0.683971,0.634793,428.0,427.0,427.50
153,card_merch_total_14,0.678767,0.633641,427.0,426.0,426.50
190,card_zip_total_3,0.677902,0.641705,426.0,429.0,427.50
...,...,...,...,...,...,...
408,card_zip_count_1_by_7,0.077067,0.055300,5.0,126.5,65.75
413,card_state_count_0_by_30,0.074863,0.025346,4.0,69.0,36.50
403,card_merch_count_1_by_14,0.058163,0.058756,3.0,137.5,70.25
414,card_state_count_1_by_7,0.053375,0.028802,2.0,79.5,40.75


In [67]:
rank2 = rank.sort_values('FDR rank', ascending = False)
rank2

,Field,KS,FDR,KS rank,FDR rank,Average rank
0,Fraud,1.000000,1.000000,430.0,430.0,430.00
190,card_zip_total_3,0.677902,0.641705,426.0,429.0,427.50
199,card_zip_total_7,0.686347,0.639401,429.0,428.0,428.50
144,card_merch_total_7,0.683971,0.634793,428.0,427.0,427.50
153,card_merch_total_14,0.678767,0.633641,427.0,426.0,426.50
...,...,...,...,...,...,...
378,merch_state_count_30,0.224970,0.003456,138.0,5.0,71.50
323,merch_zip_count_30,0.221165,0.003456,135.0,5.0,70.00
268,card_state_count_30,0.279891,0.002304,192.0,3.0,97.50
386,merch_state_actual/toal_30,0.162735,0.001152,57.0,1.5,29.25


In [68]:
rank3 = rank.sort_values('Average rank', ascending = False)
rank3

,Field,KS,FDR,KS rank,FDR rank,Average rank
0,Fraud,1.000000,1.000000,430.0,430.0,430.00
199,card_zip_total_7,0.686347,0.639401,429.0,428.0,428.50
144,card_merch_total_7,0.683971,0.634793,428.0,427.0,427.50
190,card_zip_total_3,0.677902,0.641705,426.0,429.0,427.50
153,card_merch_total_14,0.678767,0.633641,427.0,426.0,426.50
...,...,...,...,...,...,...
425,merch_state_count_0_by_30,0.122406,0.013825,21.0,37.5,29.25
395,Merchnum_count_0_by_30,0.122055,0.013825,20.0,37.5,28.75
47,Cardnum_actual/toal_14,0.128733,0.011521,24.0,25.5,24.75
421,merch_zip_count_1_by_14,0.121758,0.012673,19.0,30.0,24.50


In [77]:
# rank1.to_excel('rank_by_ks.xlsx', index = False)
# rank2.to_excel('rank_by_fdr.xlsx', index = False)
# rank3.to_excel('rank_by_avg.xlsx', index = False)

## III. Wrapper

In [81]:
filter_results = list(rank3.iloc[1:81,0])

In [82]:
df1 = trn_tst[filter_results]
df1.head()

,card_zip_total_7,card_merch_total_7,card_zip_total_3,card_merch_total_14,card_merch_total_3,card_state_total_3,card_zip_total_14,card_state_total_7,card_zip_total_1,card_merch_total_1,...,merch_state_max_7,Merchnum_max_7,card_merch_avg_3,card_zip_avg_7,Cardnum_avg_1,merch_zip_max_7,merch_state_avg_0,Merchnum_avg_0,merch_zip_avg_0,card_zip_avg_3
Recnum,,,,,,,,,,,,,,,,,,,,,
3345,1960.27,1960.27,333.47,1960.27,333.47,333.47,1960.27,1960.27,333.47,333.47,...,983.66,983.66,333.470000,653.423333,517.006667,983.66,333.470,333.470,333.470,333.470000
3346,341.23,324.43,178.41,499.22,165.41,178.41,530.02,341.23,4.37,4.37,...,27.00,27.00,5.169062,4.806056,330.435000,27.00,4.370,4.370,4.370,5.406364
3347,582.94,582.94,582.94,2306.98,582.94,955.00,2306.98,955.00,582.94,582.94,...,278.73,278.73,194.313333,194.313333,414.868571,278.73,271.930,271.930,271.930,194.313333
3348,344.85,328.05,182.03,502.84,169.03,182.03,533.64,344.85,7.99,7.99,...,27.00,27.00,5.122121,4.789583,221.496667,27.00,3.995,3.995,3.995,5.353824
3349,16.28,16.28,16.28,16.28,16.28,16.28,16.28,16.28,16.28,16.28,...,369.47,369.47,16.280000,16.280000,16.280000,369.47,16.280,16.280,16.280,16.280000


In [83]:
labels = trn_tst.Fraud.values

### Forward selection

In [88]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

In [93]:
def forward_selection(df, labels, n_features, score = 'accuracy'):

    sfs = SFS(LogisticRegression(),
              k_features = n_features,
              forward = True,
              floating = False,
              scoring = score,
              n_jobs = -1,
              verbose = 0,
              cv = 2)
    sfs.fit(df, labels)
    
    return list(sfs.k_feature_names_)

In [100]:
start = timeit.default_timer()
features = forward_selection(df1, labels, 25)
print('Total run time: {}mins'.format((timeit.default_timer() - start)/60))

Total run time: 41.775015267616645mins


In [101]:
features

['card_merch_total_3',
 'card_state_total_3',
 'card_zip_total_30',
 'card_zip_max_14',
 'card_state_total_0',
 'card_state_max_14',
 'merch_state_total_1',
 'Merchnum_total_1',
 'Cardnum_total_7',
 'card_state_max_1',
 'merch_state_total_3',
 'Cardnum_total_1',
 'Cardnum_total_0',
 'merch_zip_max_1',
 'merch_state_max_1',
 'card_zip_max_0',
 'card_merch_max_0',
 'merch_zip_max_3',
 'Cardnum_max_0',
 'Cardnum_max_1',
 'Cardnum_total_14',
 'Merchnum_max_7',
 'Cardnum_avg_1',
 'merch_state_avg_0',
 'merch_zip_avg_0']

In [106]:
final = df[features].copy()
final['Label'] = df.Fraud.values

In [109]:
final

,card_merch_total_3,card_state_total_3,card_zip_total_30,card_zip_max_14,card_state_total_0,card_state_max_14,merch_state_total_1,Merchnum_total_1,Cardnum_total_7,card_state_max_1,...,card_merch_max_0,merch_zip_max_3,Cardnum_max_0,Cardnum_max_1,Cardnum_total_14,Merchnum_max_7,Cardnum_avg_1,merch_state_avg_0,merch_zip_avg_0,Label
Recnum,,,,,,,,,,,,,,,,,,,,,
1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.620,3.6200,3.6200,0
2,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.420,31.4200,31.4200,0
3,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.490,178.4900,178.4900,0
4,3.62,3.62,3.62,3.62,3.62,3.62,7.24,7.24,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.620,3.6200,3.6200,0
5,7.24,7.24,7.24,3.62,7.24,3.62,10.86,10.86,7.24,3.62,...,3.62,3.62,3.62,3.62,7.24,3.62,3.620,3.6200,3.6200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96749,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,...,84.79,84.79,84.79,84.79,84.79,84.79,84.790,84.7900,84.7900,0
96750,118.75,886.25,718.60,599.85,118.75,1385.00,118.75,118.75,3140.25,118.75,...,118.75,118.75,685.00,685.00,8089.20,118.75,401.875,118.7500,118.7500,0
96751,363.56,435.41,363.56,363.56,363.56,363.56,5780.89,5780.89,2578.13,363.56,...,363.56,2487.98,363.56,1065.73,2578.13,2487.98,500.380,1445.2225,1445.2225,0


In [113]:
final[final.Label == 0].describe()

,card_merch_total_3,card_state_total_3,card_zip_total_30,card_zip_max_14,card_state_total_0,card_state_max_14,merch_state_total_1,Merchnum_total_1,Cardnum_total_7,card_state_max_1,...,card_merch_max_0,merch_zip_max_3,Cardnum_max_0,Cardnum_max_1,Cardnum_total_14,Merchnum_max_7,Cardnum_avg_1,merch_state_avg_0,merch_zip_avg_0,Label
count,95338.00000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,...,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.000000,95338.0
mean,488.07145,593.410568,832.334286,460.838069,469.605041,571.939229,944.240212,944.263395,2102.773251,426.217365,...,393.597011,623.085614,468.741477,575.849405,3464.427756,765.250084,377.455587,377.915949,377.905150,0.0
std,1055.41038,1222.104833,1757.273232,840.140217,979.068912,995.474904,1733.877845,1733.902678,4634.043709,810.224214,...,777.397889,1027.379365,875.741484,1007.894905,7577.539394,1177.898706,608.726017,712.439718,712.445291,0.0
min,0.01000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.140000,0.010000,...,0.010000,0.010000,0.010000,0.010000,0.140000,0.010000,0.010000,0.010000,0.010000,0.0
25%,54.12000,64.950000,103.980000,50.000000,48.320000,66.042500,111.852500,111.852500,288.000000,41.000000,...,35.980000,87.000000,47.700000,67.920000,549.000000,119.000000,53.742500,38.760000,38.760000,0.0
50%,175.00000,212.135000,314.855000,188.000000,165.000000,253.950000,374.580000,374.610000,967.705000,162.000000,...,144.965000,277.855000,188.000000,259.945000,1697.230000,372.300000,185.000000,155.000000,155.000000,0.0
75%,521.39500,638.552500,876.940000,539.907500,500.000000,713.877500,1065.000000,1065.000000,2513.107500,489.757500,...,440.000000,800.000000,549.000000,724.117500,4004.937500,1074.520000,465.791250,440.000000,440.000000,0.0
max,77100.00000,77100.000000,77100.000000,47900.000000,51000.000000,47900.000000,77100.000000,77100.000000,310902.060000,47900.000000,...,47900.000000,47900.000000,47900.000000,47900.000000,313995.060000,47900.000000,25500.000000,47900.000000,47900.000000,0.0


In [114]:
final[final.Label == 1].describe()

,card_merch_total_3,card_state_total_3,card_zip_total_30,card_zip_max_14,card_state_total_0,card_state_max_14,merch_state_total_1,Merchnum_total_1,Cardnum_total_7,card_state_max_1,...,card_merch_max_0,merch_zip_max_3,Cardnum_max_0,Cardnum_max_1,Cardnum_total_14,Merchnum_max_7,Cardnum_avg_1,merch_state_avg_0,merch_zip_avg_0,Label
count,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,...,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.0
mean,12959.348848,13900.585921,15017.964495,3499.141492,8136.597432,3800.073872,14863.996242,14863.996242,27705.128924,3306.836874,...,2850.676582,3418.039575,3150.774684,3763.926789,31114.259849,3572.245619,2015.485364,1997.315049,1996.986889,1.0
std,35318.834534,35196.530046,35243.616092,5365.723983,22150.439302,5520.352456,35496.056542,35496.056542,45661.492660,5416.652258,...,4390.257193,5394.342204,4539.944643,5694.556193,46446.773323,5419.576808,2359.639112,2566.296483,2566.494108,0.0
min,3.520000,3.520000,8.000000,3.520000,0.220000,3.520000,3.520000,3.520000,3.520000,3.520000,...,0.220000,3.520000,0.220000,3.520000,3.520000,3.520000,3.520000,0.220000,0.220000,1.0
25%,985.000000,1259.615000,1772.945000,845.690000,633.020000,870.725000,1150.265000,1150.265000,3017.110000,666.965000,...,501.130000,825.090000,600.910000,774.780000,4373.680000,858.520000,502.172500,458.039462,453.453333,1.0
50%,3677.000000,4076.050000,5096.760000,2096.450000,2453.400000,2295.000000,4472.150000,4472.150000,9885.750000,1938.060000,...,1530.820000,2088.760000,1858.420000,2087.820000,12846.610000,2111.520000,1236.537727,1050.802000,1050.802000,1.0
75%,8187.880000,10963.905000,12464.320000,4278.835000,6971.430000,4957.975000,14292.615000,14292.615000,30271.620000,4045.525000,...,3789.070000,4015.000000,4565.360000,5517.780000,38005.405000,4242.245000,2405.490000,2332.303462,2332.303462,1.0
max,306633.410000,306633.410000,306633.410000,30372.460000,217467.180000,30372.460000,306633.410000,306633.410000,312616.060000,30372.460000,...,30372.460000,30372.460000,30372.460000,30372.460000,312616.060000,30372.460000,28392.840000,28392.840000,28392.840000,1.0


In [110]:
# final.to_csv('selected_data.csv')